# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [72]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from scipy.stats import linregress
from sklearn import datasets
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [73]:
weather_data_df = pd.read_csv('weatherData.csv')
weather_data_df

,City,Cloudiness,Country,Date,Temperature,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Saint Anthony,75,US,1583369547,36.57,59,45.02,-93.22,37.99,8.05
1,Atuona,58,PF,1583369548,81.30,69,-9.80,-139.03,81.30,10.22
2,Stornoway,29,GB,1583369548,33.89,81,58.21,-6.39,34.00,4.70
3,Hilo,90,US,1583369543,71.98,69,19.73,-155.09,78.80,16.11
4,Punta Arenas,20,CL,1583369545,57.11,71,-53.15,-70.92,57.20,9.17
5,Rikitea,42,PF,1583369548,77.63,72,-23.12,-134.97,77.63,17.00
6,Mataura,0,NZ,1583369548,61.32,54,-46.19,168.86,61.32,1.72
7,Morondava,76,MG,1583369548,78.62,66,-20.28,44.28,78.62,5.79
8,Ayagoz,4,KZ,1583369548,9.46,54,47.97,80.44,9.46,3.87
9,Lipetsk,90,RU,1583369549,33.80,100,52.60,39.57,33.80,11.18


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [74]:
gmaps.configure(api_key=g_key)

locations = weather_data_df[['Lat','Lng']]

humidity = weather_data_df['Humidity'].astype(float)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=5, point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [75]:
new_types_df = pd.DataFrame(weather_data_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_types_df['Max Temp'] <= 90) & (new_types_df['Max Temp'] > 50)
wind_speed = new_types_df["Wind Speed"] < 20
cloudiness = new_types_df["Cloudiness"] < 35

new_types_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
4,Punta Arenas,57.20,9.17,20
6,Mataura,61.32,1.72,0
12,High Rock,77.00,8.05,20
17,San Quintín,62.20,11.41,5
18,Hermanus,64.00,1.88,9
20,Benguela,77.54,3.29,18
22,Bakel,80.47,8.52,0
23,Camacha,60.80,8.05,20


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [86]:
hotel_df = pd.DataFrame(weather_data_df, columns=['City','Country','Lat','Lng'])
hotel_df['Hotel Name'] = ''
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Saint Anthony,US,45.02,-93.22,
1,Atuona,PF,-9.80,-139.03,
2,Stornoway,GB,58.21,-6.39,
3,Hilo,US,19.73,-155.09,
4,Punta Arenas,CL,-53.15,-70.92,
5,Rikitea,PF,-23.12,-134.97,
6,Mataura,NZ,-46.19,168.86,
7,Morondava,MG,-20.28,44.28,
8,Ayagoz,KZ,47.97,80.44,
9,Lipetsk,RU,52.60,39.57,


In [87]:
target_coordinates = f'{hotel_df["Lat"][0]},{hotel_df["Lng"][0]}'
target_search = 'hotel'
target_radius = 5000
target_type = 'hotel'

params = {
    'location': target_coordinates,
    'keyword': target_search,
    'radius': target_radius,
    'type': target_type,
    'key': g_key   
}

base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

response = requests.get(base_url, params=params)

In [88]:

pprint(response.json(), depth=1)

{'html_attributions': [],
 'next_page_token': 'CrQCIwEAAFsSMJNlIvkcLuFigkKgLhgZw4MVwjNYtf6_Lcr_4rtnmXlN-9TRsLggnjxxpDi6eZrlfr_oACSAC6T85_qhM5tLm8NkVRN31EclSXBddXuEmCet7IfPPggmh-yd2MwMITpYuy_DkvLGblVjI5s_A70_td0yUQg-B53ny6ki9N_x01-NBdbu7ErkHaSqV2asl2d4I97SMKBFweOALibzIWwYU6_MAcBve7PTP09OfII10tI3pcQCw4DZ77Q9Jg81FcQuUjRW21Y-JXtib51RJBgwZnqE3vs-3ZtHBqhDk3xIk8GDa5rJPAUM63kwrru8hiAZy9HBkrkw1y3W5sXOSL9ValVpnUKiMQMLEVSuDldOU3QCnN_DyaNzZ_TqmIi-6TpYZIFT9jiuBA6LZ41amP8SEAGCOWTeluFqoqbz50GEOt4aFBbg4f6Uu_jhP-hKDtRq8tU8D-mt',
 'results': [...],
 'status': 'OK'}


In [89]:
hotel_df.iloc[0][0]

'Saint Anthony'

In [90]:
hotel_name = []

params = {
    'radius': 5000,
    'types': 'hotel',
    'keyword': 'hotel',
    'key': g_key
}

for indext, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']

    params['location'] = f"{lat},{lng}"
    
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()
    
    try:
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [91]:
hotel_name

['DoubleTree Suites by Hilton Hotel Minneapolis', 'Pension Temetiu Village', 'The Crown Inn Stornoway', 'Hilo Hawaiian Hotel', 'Hotel Cabo de Hornos', "Pension Maro'i", 'Falls Hotel', 'Palissandre Cote Ouest resort & SPA', 'Ayagoz', 'Mercure Lipetsk Center', 'Fema Lodge', 'Hotel Grand Chancellor Hobart', 'Hideaway Bahamas Beach Club', nan, 'Shelikof Lodge', 'Huadu Hotel', 'Bredasdorp Country Manor', "Don Eddie's Landing", 'The Marine Hermanus', 'Mary Hotel', 'Hotel Praia Morena', 'Hotel Vila Gale Touros', 'Villa La Vallée', 'Hotel Porto Santo & Spa']

In [94]:
hotel_df['Hotel Name'] = hotel_name
hotel_df.dropna()
hotel_df.to_csv(hotel_info.csv)

NameError: name 'hotel_info' is not defined

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [93]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))